In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
"""
Testing environment with stable_baselines3 library
"""
import sys
sys.path.append('../')

from stable_baselines3.common.env_checker import check_env
from envs.MO_single_agent_battery_storage_env import BatteryStorageInGridEnv

env = BatteryStorageInGridEnv()

check_env(env)

step:  2
gen max production:  [5.66733127 5.26374501 9.78413191 8.34193547 6.08912249 7.23599203
 8.1578017  5.11183249 8.22014307 5.96208632]
battery max charge:  [-1.14343886 -1.44001314 -1.39750235 -0.84089398 -0.58396823]
battery max discharge:  [1.00445797 0.81122218 1.25739356 1.04867224 1.6072459 ]
battery discharge costs:  [0.7155114  1.77165381 3.86353871 2.58455701 0.74052256]
battery charge costs:  [3.258074   2.41418756 0.26584109 3.82711663 2.12187719]
gen costs:  [2.82462455 4.10669466 1.103719   5.30170078 3.58151781 1.72810167
 3.82847644 3.55945028 2.29436813 1.18573393]
load:  40.32722327551841
status:  optimal
x value:  [-3.25695611e-13 -5.63779062e-13  4.59717293e-13 -6.53196466e-13
 -5.17615572e-13  6.99548821e-14 -6.41874412e-13 -4.91793824e-13
  7.61653113e-15  1.54209060e-13 -9.52865720e-02 -1.20001095e-01
 -1.16458529e-01 -7.00744982e-02 -4.86640191e-02  4.07777080e+01]


/home/ubuntu/.conda/envs/sustaingym/lib/python3.9/site-packages/stable_baselines3/common/env_checker.py:272: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


In [4]:
"""
Testing RL agent which randomly chooses actions
"""
import sys
sys.path.append('../')
from typing import List
import numpy as np
import os
import pandas as pd
import gym

from tqdm import tqdm
import matplotlib.pyplot as plt

env = BatteryStorageInGridEnv()

episodes = 100

rewards_lst_1 = []

for i in tqdm(range(episodes)):
    ob = env.reset()
    done = False
    rewards = np.zeros(env.MAX_STEPS_PER_EPISODE)

    while not done:
        # random action as policy
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        rewards[env.count - 1] = reward
    rewards_lst_1.append(np.sum(rewards))

print(rewards_lst_1)
# plot episode # versus total episode reward
plt.bar(list(range(episodes)), rewards_lst_1, width=0.5)

# naming the x axis 
plt.xlabel('episode #') 
# naming the y axis 
plt.ylabel('total reward')

plt.show()

  0%|          | 0/100 [00:00<?, ?it/s]

step:  2
gen max production:  [8.66358872 8.56800702 8.87879066 8.62360383 5.92778551 9.60471256
 9.01439429 6.56156133 7.34287954 5.94872275]
battery max charge:  [-0.71236938 -0.63008256 -1.37315452 -1.37413901 -1.32276758]
battery max discharge:  [1.60905689 1.74649653 1.98862637 1.81626206 1.07056664]
battery discharge costs:  [1.23260519 3.11798472 2.04318396 0.38341558 0.59263825]
battery charge costs:  [5.22835815 2.22068464 5.20830993 3.50295301 0.59263825]
gen costs:  [1.04494771 1.31491049 0.35302705 1.3091635  4.27759924 4.16047849
 0.62003643 1.22059934 4.2729571  4.56337223]
load:  40.32722327551841
status:  optimal
x value:  [ 2.73775218e-13  3.03210932e-13  4.25024761e-12  3.12299495e-13
 -5.31626116e-13 -7.59534702e-13  9.69846431e-13  1.67209788e-13
 -6.10300327e-13 -4.89809339e-13 -5.93641154e-02 -5.25068801e-02
 -1.14429543e-01 -1.14511584e-01 -1.10230631e-01  4.07782660e+01]
step:  3
gen max production:  [8.66358872 8.56800702 8.87879066 8.62360383 5.92778551 9.6047

  1%|          | 1/100 [00:00<01:30,  1.09it/s]

status:  optimal
x value:  [ 3.17877007e-10  1.28884395e-13  8.99551294e-12  1.11131796e-11
 -1.18153481e-13  3.28629512e-10  4.10781209e-11  9.87351170e-12
  2.05859824e-13 -9.85306606e-14 -5.93641154e-02 -5.25068801e-02
 -1.14429543e-01 -1.14511584e-01 -1.10230630e-01  3.21174893e+01]
step:  263
gen max production:  [8.66358872 8.56800702 8.87879066 8.62360383 5.92778551 9.60471256
 9.01439429 6.56156133 7.34287954 5.94872275]
battery max charge:  [-0.71236938 -0.63008256 -1.37315452 -1.37413901 -1.32276758]
battery max discharge:  [1.60905689 1.74649653 1.98862637 1.81626206 1.07056664]
battery discharge costs:  [1.46738161e+02 2.15847782e+03 2.16246108e+03 2.96581587e+00
 1.39449561e+00]
battery charge costs:  [874.26514678 591.29665089 139.50027282  38.87960566   1.39449561]
gen costs:  [1.41838278e+00 1.34943003e+03 3.44682208e+01 3.35789515e+01
 1.26726067e+04 1.36795681e+00 1.06130913e+01 3.68549506e+01
 1.06390581e+03 8.90090504e+03]
load:  31.94050587479765
status:  optimal
x

  1%|          | 1/100 [00:01<02:36,  1.59s/it]

step:  160
gen max production:  [8.66358872 8.56800702 8.87879066 8.62360383 5.92778551 9.60471256
 9.01439429 6.56156133 7.34287954 5.94872275]
battery max charge:  [-7.12369384e-01 -6.30082562e-01 -1.49213975e-12 -1.37413901e+00
 -2.33200126e-09]
battery max discharge:  [1.60905689 1.74649653 1.98862637 1.81626206 1.07056664]
battery discharge costs:  [2.46924824e+04 4.23715475e+03 2.75677862e+04 7.05737900e+00
 4.37419796e+00]
battery charge costs:  [9.96487625e+04 6.04539780e+02 1.42485629e+04 8.47214811e+01
 4.37419796e+00]
gen costs:  [4.17004308e+02 3.69741674e+05 5.88068210e+01 3.74172739e+03
 2.54052380e+06 2.70193547e+01 1.70075640e+02 2.78080965e+02
 4.81627882e+05 1.03216368e+06]
load:  35.17840802045258
status:  optimal
x value:  [ 4.48989023e-11  4.30598958e-13  2.99709653e-10  5.67282039e-12
 -4.66053460e-13  6.95905315e-10  1.17785155e-10  7.24199538e-11
  4.72083504e-13  3.46573515e-13 -5.93641154e-02 -5.25068801e-02
  7.97847898e-13 -1.14511583e-01  3.25698610e-09  3.

SolverError: Solver 'ECOS' failed. Try another solver, or solve with verbose=True for more information.